In [1]:
from kfp.v2 import compiler
from kfp.v2.dsl import component, pipeline

In [2]:
BUCKET_NAME = 'gs://mitochondrion-bucket-sg'
PIPELINE_ROOT = f'{BUCKET_NAME}/pipelines'

In [3]:
@component(base_image='python:3.9')
def generate_greeting(name: str) -> str:

    return f'Hello, {name}!'

In [4]:
@component(base_image='python:3.9')
def tell_day() -> str:

    from datetime import datetime

    return f'Today is {datetime.today().isoformat()}'

In [5]:
@component(base_image='python:3.9')
def combine(greeting: str, day: str) -> str:

    return f'{greeting} \n{day}'

In [6]:
@pipeline(
    name='hello-world',
    pipeline_root=PIPELINE_ROOT
)
def hello_pipeline(name: str ='world'):
    greeting = generate_greeting(name=name)
    day = tell_day()
    combine(greeting.output, day.output)

In [7]:
REGION = 'asia-southeast1'
PROJECT_ID = 'mitochondrion-project-344303'

PIPELINE_NAME = 'hello-world-pipeline'
JSON_FILE = 'hello_pipeline.json'

In [8]:
compiler.Compiler().compile(
    pipeline_func=hello_pipeline,
    package_path=JSON_FILE
)

/home/mitbal/miniconda3/lib/python3.9/site-packages/kfp/v2/compiler/compiler.py:1263: FutureWarning: APIs imported from the v1 namespace (e.g. kfp.dsl, kfp.components, etc) will not be supported by the v2 compiler since v2.0.0
  warnings.warn(


TypeError: The pipeline argument "name" is viewed as an artifact due to its type "None". And we currently do not support passing artifacts as pipeline inputs. Consider type annotating the argument with a primitive type, such as "str", "int", "float", "bool", "dict", and "list".

In [ ]:
from google.cloud import aiplatform

job = aiplatform.PipelineJob(
    display_name = PIPELINE_NAME,
    template_path = JSON_FILE,
    # template_path = 'gs://mitochondrion-bucket-sg/marketing/marketing_custom_pipeline.json',
    enable_caching = True,
    project = PROJECT_ID,
    location = REGION,
    parameter_values = {
        'name': 'Iqbal'
    }
)

# job.submit()
job.submit(service_account='default@mitochondrion-project-344303.iam.gserviceaccount.com')

ValueError: The pipeline parameter current_date is not found in the pipeline job input definitions.